In [28]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import preprocessing #to transform the feature labels
from sklearn.feature_extraction import DictVectorizer #to turn categorial variables into numeric arrays
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
train_set = pd.read_csv("Lab5_train.csv")
test_set = pd.read_csv("Lab5_test.csv")
train_set.head()

,PassengerID,Survived,Pclass,Name,Sex,Age,SibSp,ParCh,Ticket,Fare,Cabin,Embarked
0,1302,0,3,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.2250,NaN,C
1,369,0,2,"Chapman, Mr. John Henry",male,37.0,1,0,SC/AH 29037,26.0000,NaN,S
2,1128,0,3,"Petroff, Mr. Nedelio",male,19.0,0,0,349212,7.8958,NaN,S
3,491,0,2,"Mack, Mrs. (Mary)",female,57.0,0,0,S.O./P.P. 3,10.5000,E77,S
4,1087,0,3,"Olsson, Miss. Elina",female,31.0,0,0,350407,7.8542,NaN,S


Checking majority in train set

In [4]:
len(train_set[train_set["Survived"] == 1])/len(train_set) * 100

38.68852459016394

In [5]:
len(train_set[train_set["Survived"] == 0])/len(train_set) * 100

61.31147540983607

So, approx 61% didnt survive and 39% survived.

Extract Features: Using Pclass, Sex, Age, SubSp, parCh to train SVM
Adding additional features Fare and Embarked

In [14]:
train_set_features = train_set[['Pclass', 'Sex', 'Age','Survived']]
test_set_features = test_set[['Pclass', 'Sex', 'Age']]

In [18]:
train_set_features['FamilySize'] = train_set['SibSp'] + train_set['ParCh']
test_set_features['FamilySize'] = test_set['SibSp'] + test_set['ParCh']

/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
train_set_features['Age'].fillna(train_set_features['Age'].mean(), inplace=True)
test_set_features['Age'].fillna(test_set_features['Age'].mean(), inplace=True)

/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [20]:
target = train_set_features.Survived

Encoding categorical features

In [21]:
vec = DictVectorizer()
train_set_non_class = vec.fit_transform(train_set_features[['Pclass', 'Sex', 'FamilySize']].to_dict('records')).toarray()
test_set_non_class = vec.fit_transform(test_set_features[['Pclass', 'Sex', 'FamilySize']].to_dict('records')).toarray()

Scale feature values

In [22]:
min_max_scaler = preprocessing.MinMaxScaler()
train_set_age_scaled = min_max_scaler.fit_transform(train_set_features.Age)
test_set_age_scaled = min_max_scaler.fit_transform(test_set_features.Age)

/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X

In [23]:
train_set_age_scaled_non_class = np.concatenate((train_set_non_class, np.reshape(train_set_age_scaled,(915,1))), axis=1)
test_set_age_scaled_non_class = np.concatenate((test_set_non_class, np.reshape(test_set_age_scaled,(394,1))), axis=1)

Train machine learning classifier

In [24]:
#Use cross validation to improve score
C_range = range(-5, 16)
gamma_range = range(-15, 4)
best_score = 0
best_C = 0
best_gamma = 0
best_model = svm.SVC(kernel='rbf')
for C in C_range:
    for gamma in gamma_range:
        clf = svm.SVC(kernel='rbf', C=2**C, gamma=2**gamma)
        scores = cross_val_score(clf, train_set_age_scaled_non_class, target, cv=5)
        score = scores.mean()
        if best_score < score:
            best_score = score
            best_C = C
            best_gamma = gamma
            best_model = clf
print(best_score, best_C, best_gamma)

0.811983674439 9 -4


In [31]:
clf1 = svm.SVC(kernel='rbf', C=2**best_C, gamma=2**best_gamma) # best c is 1, best gamma is -6
clf2 = MLPClassifier(solver='lbfgs', activation='logistic')
clf3 = RandomForestClassifier(criterion='gini', 
                             n_estimators=700,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)


In [32]:
eclf = VotingClassifier(estimators=[('svc', clf1), ('mlp', clf2), ('dc', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['SVM', 'Neural Network', 'Random Forest', 'Ensemble']):
   scores = cross_val_score(clf, train_set_age_scaled_non_class, target, cv=5, scoring='accuracy')
   print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.81 (+/- 0.01) [SVM]
Accuracy: 0.80 (+/- 0.01) [Neural Network]
Accuracy: 0.81 (+/- 0.01) [Random Forest]
Accuracy: 0.81 (+/- 0.01) [Ensemble]


In [ ]:
eclf.fit(train_set_age_scaled_non_class, target)
target_predicted = eclf.predict(test_set_age_scaled_non_class)

Write to csv

In [22]:
output = pd.DataFrame({"PassengerId": test_set.PassengerID.values, 
              "Survived": target_predicted})
output.to_csv("submission_family_size.csv", index=False)